Data Cleaning:

-Separating the States and their corresponding Counties into Separate Columns, Separate Rows.

-Separate other categorical variables into separate rows.  

-Replace 'Unknown' with np.nan.

-Rectify string (punctunation marks, etc).

In [3]:
import pandas as pd
import numpy as np
import numpy
import re

from datetime import datetime
dateparse=lambda i: datetime.strptime(i, '%m/%d/%Y')
df=pd.read_excel('https://github.com/YounaRhymesWithTuna/My_Tableau_Viz_Links/blob/main/Electric%20Emergency%20and%20Disturbance%20Report%20(Jan.%202020%20-%20Aug.%202022)/Compiled_Annual_Summary.xls?raw=true' , parse_dates=['Date Event Began'],date_parser=dateparse)
df[75:80]

,Index,Month,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Time Took For Restoration (hrs),Area Affected,NERC Region,Alert Criteria,Event Type,Demand Loss (MW),Number of Customers Affected
75,76,March,2022-03-30,15:45:00,Unknown,Unknown,Unknown,Louisiana: Mississippi: Arkansas:,SERC,"Loss of electric service to more than 50,000 c...",Severe Weather,Unknown,66448
76,77,March,2022-03-31,08:59:00,04/01/2022,19:30:00,10.516667,"Michigan: Huron County, Tuscola County, St. Cl...",RF,"Loss of electric service to more than 50,000 c...",Severe Weather,60,70000
77,78,March,2022-03-31,22:43:00,Unknown,Unknown,Unknown,Montana: Toole County; Florida: Palm Beach Cou...,WECC/SERC,Cyber event that could potentially impact elec...,Cyber Event,0,0
78,79,March,2022-03-31,00:04:00,03/31/2022,09:20:00,9.266667,Georgia: Alabama: Mississippi: Florida:,SERC,"Loss of electric service to more than 50,000 c...",Severe Weather,157,22000
79,80,April,2022-04-05,04:40:00,04/05/2022,17:21:00,12.683333,Missouri: Jackson County;,SERC/RF,"Unexpected Transmission loss within its area, ...",System Operations,73,16299


In [32]:
df.replace('Unknown',numpy.NaN, inplace=True)


In [33]:
df['Area Affected']=df['Area Affected'].str.replace(':', ',').str.replace(';', ',')


In [34]:
# Remove if the last character of the string is " " or a comma:
df['Area Affected']=df['Area Affected'].str.rstrip(' ').str.rstrip(',')


In [35]:
# Marking each states with a * within a row to separate state and its counties from another states and their counties:
state=["Alabama",'Alaska','Arizona','Arkansas', 'California','Colorado','Connecticut','Delaware','Florida','Georgia',
'Hawaii','Idaho','Illinois','Indiana','Iowa' ,'Kansas','Kentucky', 'Louisiana','Maine','Maryland','Massachusetts',
'Michigan', 'Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico',
'New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina',
'South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin',
'Wyoming','District of Columbia']

def clean_area_affected(x):  
    area=['empty']
    for i in re.split(',',x):  
        if i.strip() in state:                           
            area.extend(( '*',i.strip(),','))
        else:            
            area.extend((i.strip(),','))
    line=''.join([str(k) for k in area])                             
    return line
    


In [36]:
df['Area Affected']=df['Area Affected'].apply(clean_area_affected)


In [37]:
# Removing if ' ' and comma are the last character of the string:
df['Area Affected']=df['Area Affected'].str.rstrip(' ').str.rstrip(',')


In [38]:
# Separate each State and its counties from other State and its counties into separate rows:
df=df.assign( Area_Affected=df['Area Affected'].str.split('*')).explode('Area_Affected').reset_index()


In [39]:
df=df.drop(df.loc[df['Area_Affected']== 'empty'].index, axis=0) 


In [40]:
df = df.reset_index(drop=True)
df.head()

,index,Index,Month,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Time Took For Restoration (hrs),Area Affected,NERC Region,Alert Criteria,Event Type,Demand Loss (MW),Number of Customers Affected,Area_Affected
0,0,1,January,2022-01-01,12:36:00,01/01/2022,14:12:00,1.600000,"empty*Washington,Thurston County",WECC,Damage or destruction of a Facility within its...,System Operations,0,0,"Washington,Thurston County"
1,1,2,January,2022-01-01,12:36:00,01/02/2022,19:43:00,7.116667,"empty*Washington,Thurston County",WECC,Damage or destruction of a Facility within its...,System Operations,0,0,"Washington,Thurston County"
2,2,3,January,2022-01-01,01:01:00,NaN,NaN,NaN,"empty*Georgia,Clayton County",SERC,Damage or destruction of its Facility that res...,Vandalism,0,0,"Georgia,Clayton County"
3,3,4,January,2022-01-01,00:06:00,01/01/2022,05:51:00,5.750000,"empty*New Hampshire,Merrimack County",NPCC,Damage or destruction of its Facility that res...,Suspicious Activity,2,845,"New Hampshire,Merrimack County"
4,4,5,January,2022-01-02,13:21:00,01/02/2022,17:38:00,4.283333,"empty*California,Tuolumne County",WECC,Electrical System Separation (Islanding) where...,Severe Weather,3,1706,"California,Tuolumne County"


In [41]:
# Get rid of the comma in string[-1] position in Area_Affected column:
df['Area_Affected']=df['Area_Affected'].str.rstrip(',')


In [42]:
# Separate the State from the Counties (still in the same row):
df['State']=df['Area_Affected'].str.split(',', 1, expand=True)[0]
df['counties']=df['Area_Affected'].str.split(',',1,expand=True)[1]


In [43]:
df.State.unique()

array(['Washington', 'Georgia', 'New Hampshire', 'California',
       'South Carolina', 'North Carolina', 'Alabama', 'Mississippi',
       'Massachusetts', 'Florida', 'Texas', 'Indiana', 'Kentucky', 'Ohio',
       'Pennsylvania', 'New Mexico', 'New York', 'Maine', 'Arkansas',
       'Louisiana', 'Nevada', 'Colorado', 'Connecticut', 'Rhode Island',
       'Vermont', 'Nebraska', 'West Virginia', 'Tennessee', 'Illinois',
       'Minnesota', 'New Jersey', 'Maryland', 'South Dakota', 'Michigan',
       'Montana', 'Missouri', 'Oregon', 'Iowa', 'North Dakota', 'Arizona',
       'Idaho', 'Utah', 'Kansas', 'Wyoming', 'Virginia', 'Wisconsin',
       'Oklahoma', 'District of Columbia', 'Delaware',
       'emptyNorthern and Central California', 'emptyWestern NY',
       'emptyCentral Oklahoma', 'emptyTexas panhandle'], dtype=object)

In [44]:
df.loc[df['State']=='emptyTexas panhandle']

,index,Index,Month,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Time Took For Restoration (hrs),Area Affected,NERC Region,Alert Criteria,Event Type,Demand Loss (MW),Number of Customers Affected,Area_Affected,State,counties
1131,892,893,August,2020-08-23,10:30:00,NaN,NaN,NaN,"emptyTexas panhandle,SE NM",TRE,Public appeal to reduce the use of electricity...,Severe Weather,NaN,NaN,"emptyTexas panhandle,SE NM",emptyTexas panhandle,SE NM


In [45]:
# Rectify State:
df.loc[df['State']=='emptyTexas panhandle', 'County'] = 'Panhandle County'
df['State']=df['State'].replace(['emptyNorthern and Central California','emptyWestern NY','emptyCentral Oklahoma','emptyTexas panhandle'],['California','New York','Oklahoma','Texas'])


In [46]:
# Before creating same index, save to CSV (will be helpful for counting states and hours etc. in Tableu):
df1=df[['Month','Time Event Began','Date Event Began','Time Took For Restoration (hrs)','Demand Loss (MW)','Number of Customers Affected','NERC Region','State','counties']]
# df1.to_csv('Count_Electric_Disturbance.csv',index=False)


In [47]:
# Separate the counties in each row into separate rows by exploding:
df=df.assign( County=df['counties'].str.split(',')).explode('County').reset_index()


In [48]:
df[50:60]

,level_0,index,Index,Month,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Time Took For Restoration (hrs),Area Affected,NERC Region,Alert Criteria,Event Type,Demand Loss (MW),Number of Customers Affected,Area_Affected,State,counties,County
50,45,32,33,January,2022-01-29,08:45:00,01/30/2022,01:00:00,16.250000,"empty*Connecticut,*Massachusetts,*New Hampshir...",NPCC,"Loss of electric service to more than 50,000 c...",Severe Weather,120,12000,Massachusetts,Massachusetts,None,None
51,46,32,33,January,2022-01-29,08:45:00,01/30/2022,01:00:00,16.250000,"empty*Connecticut,*Massachusetts,*New Hampshir...",NPCC,"Loss of electric service to more than 50,000 c...",Severe Weather,120,12000,New Hampshire,New Hampshire,None,None
52,47,32,33,January,2022-01-29,08:45:00,01/30/2022,01:00:00,16.250000,"empty*Connecticut,*Massachusetts,*New Hampshir...",NPCC,"Loss of electric service to more than 50,000 c...",Severe Weather,120,12000,Rhode Island,Rhode Island,None,None
53,48,32,33,January,2022-01-29,08:45:00,01/30/2022,01:00:00,16.250000,"empty*Connecticut,*Massachusetts,*New Hampshir...",NPCC,"Loss of electric service to more than 50,000 c...",Severe Weather,120,12000,Vermont,Vermont,None,None
54,49,32,33,January,2022-01-29,08:45:00,01/30/2022,01:00:00,16.250000,"empty*Connecticut,*Massachusetts,*New Hampshir...",NPCC,"Loss of electric service to more than 50,000 c...",Severe Weather,120,12000,Maine,Maine,None,None
55,50,33,34,January,2022-01-30,20:41:00,01/31/2022,00:42:00,4.016667,empty*Nebraska,MRO,Complete loss of monitoring or control capabil...,System Operations,0,0,Nebraska,Nebraska,None,None
56,51,34,35,January,2022-01-31,19:58:00,01/31/2022,20:10:00,0.200000,"empty*West Virginia,Monongalia County",RF,Physical threat to its Facility excluding weat...,Suspicious Activity,0,0,"West Virginia,Monongalia County",West Virginia,Monongalia County,Monongalia County
57,52,35,36,February,2022-02-01,12:13:00,02/01/2022,12:25:00,0.200000,"empty*California,Yuba County",WECC,Electrical System Separation (Islanding) where...,System Operations,10,8124,"California,Yuba County",California,Yuba County,Yuba County
58,53,36,37,February,2022-02-03,10:00:00,02/07/2022,20:00:00,10.000000,"empty*Tennessee,Shelby County",SERC,"Loss of electric service to more than 50,000 c...",Severe Weather/Transmission Interruption,NaN,255000,"Tennessee,Shelby County",Tennessee,Shelby County,Shelby County
59,54,37,38,February,2022-02-03,12:56:00,02/04/2022,20:30:00,7.566667,empty*Ohio,RF,"Loss of electric service to more than 50,000 c...",Severe Weather/Transmission Interruption,NaN,51000,Ohio,Ohio,None,None


In [49]:
df=df.drop(['Area Affected','Area_Affected','counties'],axis=1)


In [50]:
# explode columns that have multiple categorical values:
df=df.assign(NERC_Region=df['NERC Region'].str.split('/')).explode('NERC_Region')
df=df.assign(Event_Type=df['Event Type'].str.split('/')).explode('Event_Type')
df=df.assign(Alert_Criteria=df['Alert Criteria'].str.split('/')).explode('Alert_Criteria')

In [51]:
df=df.drop(['NERC Region','Event Type','Alert Criteria'], axis=1)

In [52]:
df.County.unique()

array(['Thurston County', 'Clayton County', 'Merrimack County',
       'Tuolumne County', None, 'Plymouth County', 'Citrus County',
       'Cameron County', 'Willacy County', 'Adams County',
       'Hampden County[13]', 'Bucks County', 'Potter County',
       'Orange County', 'Jackson County', 'Greenville County',
       'Harris County', 'Westmoreland County', 'Chelan County',
       'Dutchess County', 'Greene County', 'Leon County', 'Clark County',
       'Douglas County', 'Pueblo County', 'El Paso County',
       'Prowers County', 'Monongalia County', 'Yuba County',
       'Shelby County', 'Ulster County', 'Alameda County',
       'Providence County[13]', 'Lafayette Parish', 'Llano County',
       'Montgomery County', 'San Bernardino County', 'Kern County',
       'Solano County', 'San Diego County', 'Travis County',
       'Grant County', 'Mitchell County', 'Marion County',
       'Hendricks County', 'DuPage County', 'Terrell County',
       'Scotts Bluff County', 'Hennepin County',

In [53]:
# Rectify counties part 1:
df['County']=df['County'].replace(['Hampden County[13]','Providence County[13]','Suffolk County[13]','Queen Anne&#39','New Haven County[13]'],['Hampden County','Providence County','Suffolk County',"Queen Anne's County",'New Haven County'])


In [54]:
df.loc[ (df['County']=='s County') |(df['County']=='SE NM') | (df['County']=='City of[16]') ]


,level_0,index,Index,Month,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Time Took For Restoration (hrs),Demand Loss (MW),Number of Customers Affected,State,County,NERC_Region,Event_Type,Alert_Criteria
453,301,225,226,August,2022-08-04,18:00:00,08/05/2022,02:00:00,8.000000,NaN,75500,Maryland,City of[16],RF,Severe Weather,"Loss of electric service to more than 50,000 c..."
1384,908,707,708,March,2020-03-27,13:06:00,03/27/2020,13:58:00,0.866667,0,0,Maryland,s County,RF,System Operations,Complete loss of monitoring or control capabil...
1781,1131,892,893,August,2020-08-23,10:30:00,NaN,NaN,NaN,NaN,NaN,Texas,SE NM,TRE,Severe Weather,Public appeal to reduce the use of electricity...


In [55]:
# Rectify counties part 2: After comparing to the original csv, we see that the recorder made an error and did not 
#  specify the City for 'City of[16]'. The s County was due to str.split in 'Queen Anne&#39;s County' AKA 
# Queen Anne's County. Thus, we can delete them both:
df=df.drop(df.loc[df['County']== 'City of[16]'].index, axis=0)
df=df.drop(df.loc[df['County']=='s County'].index, axis=0)


In [56]:
df[100:110]

,level_0,index,Index,Month,Date Event Began,Time Event Began,Date of Restoration,Time of Restoration,Time Took For Restoration (hrs),Demand Loss (MW),Number of Customers Affected,State,County,NERC_Region,Event_Type,Alert_Criteria
87,77,53,54,March,2022-03-02,09:26:00,03/02/2022,10:05:00,0.650000,0,0,South Carolina,None,SERC,System Operations,Unplanned evacuation from its Bulk Electric Sy...
88,78,54,55,March,2022-03-05,22:58:00,03/06/2022,19:38:00,20.666667,0,0,Illinois,DuPage County,SERC,System Operations,"Unexpected Transmission loss within its area, ..."
88,78,54,55,March,2022-03-05,22:58:00,03/06/2022,19:38:00,20.666667,0,0,Illinois,DuPage County,RF,System Operations,"Unexpected Transmission loss within its area, ..."
89,79,55,56,March,2022-03-06,02:30:00,03/06/2022,17:00:00,14.500000,0,0,Texas,Cameron County,TRE,Cyber Event,Cyber event that could potentially impact elec...
90,80,56,57,March,2022-03-08,13:50:00,03/08/2022,14:39:00,0.816667,48,5855,Georgia,Terrell County,TRE,System Operations,"Unexpected Transmission loss within its area, ..."
91,81,57,58,March,2022-03-10,08:51:00,03/10/2022,15:15:00,6.400000,0,0,Nebraska,Scotts Bluff County,MRO,Transmission Interruption,"Unexpected Transmission loss within its area, ..."
92,82,58,59,March,2022-03-11,06:35:00,03/11/2022,07:00:00,0.416667,0,0,Texas,Cameron County,TRE,Suspicious Activity,Physical threat to its Facility excluding weat...
93,83,59,60,March,2022-03-12,08:33:00,NaN,NaN,NaN,139,19380,Alabama,None,SERC,Severe Weather,"Loss of electric service to more than 50,000 c..."
94,84,59,60,March,2022-03-12,08:33:00,NaN,NaN,NaN,139,19380,Georgia,None,SERC,Severe Weather,"Loss of electric service to more than 50,000 c..."
95,85,59,60,March,2022-03-12,08:33:00,NaN,NaN,NaN,139,19380,Mississippi,None,SERC,Severe Weather,"Loss of electric service to more than 50,000 c..."


In [57]:
df=df.drop(['level_0','index'],axis=1)
# df.to_csv('Final_Cleaned_Data_Electric_Disturbance.csv',index=False)
